## Module 6 Practice - Projections and Vector Maps


In this notebook, we will see how to **draw vector maps from different sources.** 

In [ ]:
# Useful libraries to visualize maps 
library(ggplot2)
library(sp)
library(maps)
library(maptools)
library(mapproj)
library(mapdata)

**Let's start with plain R to draw flights in between cities.** 

Plain R can be sometimes shorter to code. In this example, our data set has **two files**; 
 - flight information for airlines between airports, 
 - geo coordinates of the airports. 
 
 First, we will go through the flight data and **find out the coordinates of the originating and destination airports and visualize the number of flights**. 

In [ ]:
# airport codes and coordinates 
airports <- read.csv("/dsa/data/all_datasets/spatial/airports.csv", as.is=TRUE, header=TRUE)
# flight destinations and counts 
flights <- read.csv("/dsa/data/all_datasets/spatial/flights.csv", as.is=TRUE, header=TRUE)

airports$lat <- as.numeric(airports$lat)
airports$long <- as.numeric(airports$long)

# Look at the data frames 
head(airports)
head(flights)

The following code draws a world map in plain R using the `map()` function and draws flight paths on it in a loop.

In [ ]:
# Draw world map in plain R 
map("world", col="#f2f2f2", fill=TRUE, bg="white", lwd=0.05, xlim=c(-172, -57), ylim=c(12, 72))

# This is needed to compute the great circle between two locations on earth 
library(geosphere)

# Display only American Airlines 
fsub <- flights[flights$airline == "AA",]

# go through EACH flight and FIND the coordinates of originating and destination airport 
for (j in 1:length(fsub$airline)) {
    air1 <- airports[airports$iata == fsub[j,]$airport1,]
    air2 <- airports[airports$iata == fsub[j,]$airport2,]

    # compute the great circle and create a path of 100 points between endpoints 
    inter <- data.frame(gcIntermediate(c(air1[1,]$long, air1[1,]$lat), c(air2[1,]$long, air2[1,]$lat), n=100, addStartEnd=TRUE))
    
    # draw the arc for this flight; line thickness encodes number of flights 
    lines(inter, col="black", lwd=2*fsub[j,]$cnt/max(fsub$cnt))
}

---

### Maps in ggplot

**We can also read the vector map data from a shapefile**. A shapefile contains geospatial vector data that define geometric shapes of a map and the related attributes. 


Shapefiles can be obtained from online resources such as [US Census Bureau](https://www.census.gov/programs-surveys/geography/geographies/mapping-files.html). 


We can **read the shapefiles and convert them to data frames** that **ggplot** can display. But they can be very large and take some time to process. The following shows how to draw the vector maps given in shapefile format using ggplot. 

In [ ]:
# load the shapefile and convert to a data frame ggplot can use. 
us_shp <- readShapePoly("/dsa/data/all_datasets/spatial/cb_2015_us_state_500k.shp")
us_map <- fortify(us_shp)
head(us_map)

In [ ]:
# we can also the rgdal library to read shapefiles.
library(rgdal)
us_shp2 <- readOGR(dsn="/dsa/data/all_datasets/spatial/", layer="cb_2015_us_state_500k", GDAL1_integer64_policy=TRUE)
us_map2 <- fortify(us_shp2)
head(us_map2)

There are **two geoms** we can use to draw a vector map: `geom_path` and `geom_polygon`. A polygon is simply a filled path. The following draws the map using the `geom_path()` geom and sets the projection to Mercator. We also limit the map to particular coordinates using `xlim` and `ylim` for longitudes and lattidues, respectively. 

In [ ]:
# draw the map 
ggplot(us_map, aes(x=long, y=lat, group=group)) + 
geom_path() + 
coord_map("mercator") + 
xlim(c(-172, -57)) + ylim(c(12, 72)) 

In [ ]:
ggplot(us_map, aes(x=long, y=lat, group=group)) + 
geom_path() + 
coord_map("polyconic") +        # use different projection 
xlim(c(-172, -57)) + ylim(c(12, 72)) 

With a `geom_polygon`, we can set the color to fill the polygons. Later, we will make it a visual variable! 

In [ ]:
ggplot(us_map, aes(x=long, y=lat, group=group)) + 
geom_polygon(fill="lightgray", color="darkgray", size=0.2) + 
coord_map("polyconic") + 
xlim(c(-172, -57)) + ylim(c(12, 72)) 

**Let's see how to get the vector data using map_data() function of the `maps` library. It will be faster, and we will use this method to get vector map data for our maps.** 

In [ ]:
world <- map_data("world")

head(world)

In the data frame, `long` and `lat` are the coordinates of the corners of each polygon; `group` provides a unique identifier for contiguous areas within a region, and `region` stands for countries. 

If we plot this data using `geom_point`, we will see the points representing coordinates of each corner of the polygons: 

In [ ]:
w <- ggplot() + 
     geom_point(data=world, aes(x=long, y=lat), size=0.1)+
     coord_equal() +
     xlim(c(-172, -57)) + ylim(c(12, 72))  + # limit to US 
     theme_void() 
w

Using `geom_polygon`, we will see the polygons as lines drawn between corner points following the order and grouping in the map data. 

In [ ]:
w <- ggplot()
w <- w + geom_polygon(data=world, aes(x=long, y=lat, group=group), color="black", fill="#d6bf86", size=0.1)
w <- w + coord_equal() 
w <- w + theme_void()
w

This is what happens **without** grouping. 

In [ ]:
w <- ggplot()
w <- w + geom_polygon(data=world, aes(x=long, y=lat), color="black", fill="#d6bf86", size=0.1)
w <- w + coord_equal() 
w <- w + theme_void()
w

### YOUR TURN: 
**Create a world map where each country is colored separately.**

Now, the fill has to be in aes. Which variable would you use for the fill aesthetic? 



In [ ]:
< YOUR CODE HERE >

---

Similar to the world map, we can also get the US map data with state boundaries. 

In [ ]:
us <- map_data("state")

head(us)

In [ ]:
gg <- ggplot() + 
      geom_polygon(data=us, aes(x=long, y=lat, group=group), color="black", fill=NA, size=0.15) +

    # this is a good projection for US
      coord_map("polyconic") +
      theme_void() 
gg 

There is a **special geom just for this type of map data** called `geom_map` which assumes the data frame has `long`, `lat`, and `group` columns, so we don't need to define them in aes. We will see later that it is also useful for other purposes. Sometimes, using `geom_map` can be faster than using `geom_polygon`.

When you use the `geom_map`, make sure to use `expand_limits` or set `xlim` and `ylim` manually. 

In [ ]:
gg <- ggplot(data=us) + 
      geom_map(map=us, aes(map_id=region), color="black", fill=NA, size=0.15) +

# this is a good projection for US
      coord_map("polyconic") +
      expand_limits(x=us$long, y=us$lat) +
      theme_void() 
gg 

We can add some color and change the projection like this: 

In [ ]:
gg <- ggplot(data=us) + 
      geom_map(map=us, aes(map_id=region), color="white", fill="lightblue", size=0.15) + 

#This is better projection
      coord_map("albers", lat0=30, lat1=40) + 
      expand_limits(x=us$long, y=us$lat) +
      theme_void() 
gg

The following shows how we can get data for different levels of detail and use **multiple layers** to overlay them. 

In [ ]:
state <- map_data("state")
county <- map_data("county")
usa <- map_data("usa")

gg <- ggplot() +
      geom_map(data=county, map=county, aes(map_id=region), color="grey", fill=NA, size=0.1) + 

      geom_map(data=state, map=state, aes(map_id=region), color="red", fill=NA, size=0.3) +

      geom_map(data=usa, map=usa, aes(map_id=region), color="blue", fill=NA, size=0.6) + 

      coord_map("albers", lat0=30, lat1=40) + 
      expand_limits(x=usa$long, y=usa$lat) +
      theme_void()
gg

Take a look at the county map data: we can pick counties for a given state using `region` for subsetting.

In [ ]:
head(county)

In [ ]:
#Let's pick Missouri
mo <- county[which(county$region=="missouri"),]
head(mo)

Same thing could also be done like this:

In [ ]:
mo2 <- map_data("county", "missouri")
head(mo2)

Let's select the Boone county from MO. **County names are NOT unique!** There are Boone counties in other states, too, so we will combine Boone and Missouri like this:

In [ ]:
boone <- county[which(county$subregion=="boone" & county$region=="missouri"),]

In [ ]:
gg <- ggplot() + 
      geom_map(data=mo, map=mo, aes(map_id=region), color="black", fill=NA, size=0.1) + 
      geom_map(data=boone, map=boone, aes(map_id=region), color="red", fill=NA, size=0.4) + 

      coord_map("polyconic") + 
      expand_limits(x=mo$long, y=mo$lat) +
      theme_void()
gg

### YOUR TURN:

**Plot the same map as above with neighbor counties of Boone county colored blue**. (Neighbors are Audrain, Callaway, Cole, Cooper, Howard, Moniteau, Randolph counties).

In [ ]:
< YOUR CODE HERE >


---

**Let's get coordinates of some of the cities in MO and display them.** 


In [ ]:
# this data frame has all the info we need
head(us.cities)

In [ ]:
mo_cities <- subset(us.cities, country.etc=="MO")
mo_cities

In [ ]:
# This is a bubble map overlayed on the previous map we created 
gg + geom_point(data=mo_cities, aes(x=long, y=lat, size=pop, color=factor(capital))) +
scale_color_manual(values=c("blue","red"))

---

**Now, let's finish with a similar plot like in the beginning for the airline routes**, but this time using `ggplot` and `dplyr` for smarter data manipulation. 

Study the following code cells to understand how the map is constructed.

In [ ]:
# pick only busy routes 
flights <- subset(flights, cnt>300)

# get airport locations
airport_locs <- airports[, c("iata","long", "lat")]

In [ ]:
library(dplyr)

# Link airport lat long to origin and destination

OD <- left_join(flights, airport_locs, by=c("airport1"="iata"))
OD <- left_join(OD, airport_locs, by=c("airport2"="iata"))
head(OD)

In [ ]:
# Now, create curves on the map with a fixed curvature - no great circle computation 
ggplot() + 

geom_map(data=world,map=world, aes(map_id=region), fill="#e6ef86", color="black", size=0.1) +

geom_curve(data=OD, aes(x=long.x, y=lat.x, xend=long.y, yend=lat.y, color=cnt), size=0.1,
                 curvature=-0.2, arrow=arrow(length=unit(0.01, "npc"))) +
    
scale_colour_distiller(palette="Blues", guide="none") +

coord_equal() +

xlim(c(-172,-57)) + ylim(c(12,72)) + 

theme_void()

### YOUR TURN: 

Use the above code and modify it to display only the contiguous United States and add state boundaries (Hint: change the map to one of the earlier maps we have used and play with xlim and ylim).

In [ ]:
< YOUR CODE HERE > 
